In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
file_dir = 'D:/Astro/StarCatalogues'
file_name = 'Imm Deep Sky Compendium - 2024 - rev5a.xlsm'
file_path = os.path.join(file_dir, file_name)


In [ ]:
data = pd.read_excel(file_path, sheet_name="Main", skiprows=6)


In [ ]:
# Fix up the column headers

# Extract column names for columns 0 to 33, and from row 0 for columns 34 onward
header = list(data.columns[:34]) + data.iloc[0, 34:].tolist()
substitutions = {
    'Object Name & Image': 'Object',
    'Unnamed: 1': 'Simbad',
    'Unnamed: 2': 'Aladin',
    'Right Ascension': 'RA_hms',
    'Unnamed: 12': 'RA_deg',
    'Declination': 'Dec_dms',
    'Unnamed: 14': 'Dec_deg',
    'Const.': 'Constellation',
    'Nick.': 'Name',
    'Alt. ID': 'Name_Alt',
    'Surf.': 'Brightness',
    'Inclin.': 'Inclination',
    17: 'Next17',
    19: 'Next19',
    21: 'Next21',
    23: 'Next23',
    'Unnamed': 'Rename',
}
header[-6]='ObjAz'
header[-5]='Key'
header[-4]='FOV'
header = [substitutions.get(col, col) for col in header]
index = [col_idx for col_idx, col_name in enumerate(header) if col_name == 'Abell']
header[index[0]]='AbellG'   # Replace duplate Abell with AbellG
header[index[1]]='AbellN'   # Replace duplate Abell with AbellG
data.columns = header


In [ ]:
# Drop junk columns and first 2 rows (header data)
columns_to_drop = ['Simbad', 'Aladin', 'My', 'My.1', 'My.2', 'Next17', 'Next19', 'Next21', 'Next23', 'Time', 'Date', 'Alt.', 'Separ.', 'Index', 'ObjAz', 'Transit Time', 'Add. Filters', 'Sort to ']
data = data.drop(columns=columns_to_drop)
data = data.iloc[2:].reset_index(drop=True)  


In [ ]:
data['RA_deg'] = pd.to_numeric(data['RA_deg'], errors='coerce')
data['Dec_deg'] = pd.to_numeric(data['Dec_deg'], errors='coerce')
data['Size'] = pd.to_numeric(data['Size'], errors='coerce')
data['Visual'] = pd.to_numeric(data['Visual'], errors='coerce')

In [ ]:
data['NGC'] = pd.to_numeric(data['NGC'], errors='coerce').astype('Int64')
data['IC'] = pd.to_numeric(data['IC'], errors='coerce').astype('Int64')
data['H400'] = pd.to_numeric(data['H400'], errors='coerce').astype('Int64')
data['Messier'] = pd.to_numeric(data['Messier'], errors='coerce').astype('Int64')
data['Caldwell'] = pd.to_numeric(data['Caldwell'], errors='coerce').astype('Int64')
data['Arp'] = pd.to_numeric(data['Arp'], errors='coerce').astype('Int64')
data['Hickson'] = pd.to_numeric(data['Hickson'], errors='coerce').astype('Int64')
data['AbellG'] = pd.to_numeric(data['AbellG'], errors='coerce').astype('Int64')
data['AbellN'] = pd.to_numeric(data['AbellN'], errors='coerce').astype('Int64')
data['UGC'] = pd.to_numeric(data['UGC'], errors='coerce').astype('Int64')
data['PGC'] = pd.to_numeric(data['PGC'], errors='coerce').astype('Int64')
data['Griff.'] = pd.to_numeric(data['Griff.'], errors='coerce').astype('Int64')
data['Kohou.'] = pd.to_numeric(data['Kohou.'], errors='coerce').astype('Int64')
data['Mink.'] = pd.to_numeric(data['Mink.'], errors='coerce').astype('Int64')
data['Barn.'] = pd.to_numeric(data['Barn.'], errors='coerce').round().astype('Int64') # has decimals
data['Gum'] = pd.to_numeric(data['Gum'], errors='coerce').astype('Int64')
data['LBN'] = pd.to_numeric(data['LBN'], errors='coerce').astype('Int64')
data['LDN'] = pd.to_numeric(data['LDN'], errors='coerce').astype('Int64')
data['Sh2'] = pd.to_numeric(data['Sh2'], errors='coerce').astype('Int64')
#data['SNR'] = pd.to_numeric(data['SNR'], errors='coerce').astype('Int64')  # has text
data['vdB'] = pd.to_numeric(data['vdB'], errors='coerce').astype('Int64')
data['HT'] = pd.to_numeric(data['HT'], errors='coerce').astype('Int64')
data['SD'] = pd.to_numeric(data['SD'], errors='coerce').astype('Int64')
data['OB'] = pd.to_numeric(data['OB'], errors='coerce').astype('Int64')
data['SP'] = pd.to_numeric(data['SP'], errors='coerce').astype('Int64')
data['FG'] = pd.to_numeric(data['FG'], errors='coerce').astype('Int64')


In [ ]:
data.describe()

In [ ]:
data

In [ ]:
# Clean up the type column
data['Type'] = data['Type'].replace({'Neb ':'Neb'})
type_abbr = {
    'Neb': 'Nebula',
    'Gal': 'Galaxy',
    'Stars': 'Cluster',
    'Star': 'Star',
}
type_enum = {abbr: idx for idx, abbr in enumerate(type_abbr.keys())}
C1_lookup = {idx: name for idx, name in enumerate(type_abbr.values())}
data['C1'] = data['Type'].map(type_enum)
data['C1Name'] = data['C1'].map(C1_lookup)

unique_abbr = set(data['Type'].unique())
valid_abbr = set(type_abbr.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")




In [ ]:
# Clean up the constellation column
data['Constellation'] = data['Constellation'].replace({'Apu':'Aps', 'Lmi': 'LMi', 'Uma': 'UMa', 'Crux': 'Cru', 'Cent': 'Cen', 'Pis': 'Pic'})
constellation_abbr = {
    'And': 'Andromeda', 'Ant': 'Antlia', 'Aps': 'Apus', 'Aql': 'Aquila', 'Aqr': 'Aquarius',
    'Ara': 'Ara', 'Ari': 'Aries', 'Aur': 'Auriga', 'Boo': 'Boötes', 'CMa': 'Canis Major',
    'CMi': 'Canis Minor', 'CVn': 'Canes Venatici', 'Cam': 'Camelopardalis', 'Cap': 'Capricornus',
    'Car': 'Carina', 'Cas': 'Cassiopeia', 'Cen': 'Centaurus', 'Cep': 'Cepheus', 'Cet': 'Cetus',
    'Cha': 'Chamaeleon', 'Cir': 'Circinus', 'Cnc': 'Cancer', 'Col': 'Columba', 'Com': 'Coma Berenices',
    'CrA': 'Corona Australis', 'CrB': 'Corona Borealis', 'Crt': 'Crater', 'Cru': 'Crux',
    'Crv': 'Corvus', 'Cyg': 'Cygnus', 'Del': 'Delphinus', 'Dor': 'Dorado', 'Dra': 'Draco',
    'Eri': 'Eridanus', 'For': 'Fornax', 'Gem': 'Gemini', 'Gru': 'Grus', 'Her': 'Hercules',
    'Hor': 'Horologium', 'Hya': 'Hydra', 'LMi': 'Leo Minor', 'Lac': 'Lacerta', 'Leo': 'Leo',
    'Lep': 'Lepus', 'Lib': 'Libra', 'Lup': 'Lupus', 'Lyn': 'Lynx', 'Lyr': 'Lyra',
    'Men': 'Mensa', 'Mic': 'Microscopium', 'Mon': 'Monoceros', 'Mus': 'Musca', 'Nor': 'Norma',
    'Oct': 'Octans', 'Oph': 'Ophiuchus', 'Ori': 'Orion', 'Pav': 'Pavo', 'Peg': 'Pegasus',
    'Per': 'Perseus', 'Pic': 'Pictor', 'PsA': 'Piscis Austrinus', 'Psc': 'Pisces', 'Pup': 'Puppis',
    'Pyx': 'Pyxis', 'Ret': 'Reticulum', 'Scl': 'Sculptor', 'Sco': 'Scorpius', 'Sct': 'Scutum',
    'Ser': 'Serpens', 'Sex': 'Sextans', 'Sge': 'Sagitta', 'Sgr': 'Sagittarius', 'Tau': 'Taurus',
    'Tel': 'Telescopium', 'TrA': 'Triangulum Australe', 'Tri': 'Triangulum', 'Tuc': 'Tucana',
    'UMa': 'Ursa Major', 'UMi': 'Ursa Minor', 'Vel': 'Vela', 'Vir': 'Virgo', 'Vol': 'Volans',
    'Vul': 'Vulpecula'
}
const_enum = {abbr: idx for idx, abbr in enumerate(constellation_abbr.keys())}
Cn_lookup = {idx: name for idx, name in enumerate(constellation_abbr.values())}
data['Cn'] = data['Constellation'].map(const_enum)
data['CnName'] = data['Cn'].map(Cn_lookup)



unique_abbr = set(data['Constellation'].unique())
valid_abbr = set(constellation_abbr.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")
data['Constellation'].unique()

data['ConstName'] = data['Constellation'].map(constellation_abbr)

grouped = data.groupby(['Constellation', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Constellation', y='Count', color='Type', title='Object Constellation by Type', 
             category_orders={'Constellation': list(constellation_abbr.keys())} ) 
fig.show()



In [ ]:
# Clean up the type column
data['Sub'] = data['Sub'].replace({'Group ':'Group', 'Pair ': 'Pair', 'Spiral ': 'Spiral', 'Mol CLd':'Mol Cld', 'Stars':'Star' })
sub_abbr = {
    # Multiple Galaxies
    'Chain': 'Set of Chained Galaxies',
    'Cluster': 'Set of Clustered Galaxies',
    'Group': 'Set of Grouped Galaxies',
    'Merger': 'Set of Merging Galaxies',
    'Pair': 'Pair of Galaxies',
    'Trio': 'Trio of Galaxies',

    # Individual Galaxy
    'BCD': 'Blue Compact Dwarf Galaxy',
    'Coll': 'Collisional Ring Galaxy',
    'Dwarf': 'Dwarf Galaxy',
    'Ellip': 'Elliptical Galaxy',
    'Floc': 'Flocculent Galaxy',
    'Lent': 'Lenticular Galaxy',
    'Mag': 'Magellanic Galaxy',
    'Polar': 'Polar Galaxy',
    'Spiral': 'Spiral Galaxy',

    # Nebula
    'Dark': 'Dark Nebula',
    'Em': 'Emission Nebula',
    'Mol Cld': 'Molecular Cloud Nebula',
    'PN': 'Planetary Nebula',
    'PPN': 'Protoplanetary Nebula',
    'Refl': 'Reflection Nebula',
    'SNR': 'Supernova Remnant Nebula',

    # Stellar associations
    'GC': 'Globular Cluster',
    'HH': 'Herbig-Haro Object',
    'Nova': 'Nova Object',
    'OC': 'Open Cluster',
    'Star': 'Star',
    'Star Cld': 'Star Cloud',
    'YSO': 'Young Stellar Object'
}
sub_enum = {abbr: idx for idx, abbr in enumerate(sub_abbr.keys())}
C2_lookup = {idx: name for idx, name in enumerate(sub_abbr.values())}
data['C2'] = data['Sub'].map(sub_enum)
data['C2Name'] = data['C2'].map(C2_lookup)

unique_abbr = set(data['Sub'].unique())
valid_abbr = set(sub_abbr.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")


data['SubName'] = data['Sub'].map(sub_abbr)


grouped = data.groupby(['Sub', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Sub', y='Count', color='Type', title='Object Subtype by Type') 
fig.show()



In [ ]:
bins = [    0,         2,                4,                  6,                 8,              10,                  12,           3000]
labels = ['Brilliant (<2 Mag)', 'Bright (<4 Mag)', 'Visible (<6 Mag)', 'Dim (<8 Mag)', 'Faint (<10 Mag)', 'Ghostly (<12 Mag)', 'Ultra Faint (12+ Mag)']
data['VisualBin'] = pd.cut(data['Visual'], bins=bins, labels=labels, include_lowest=True)

Vz_enum = {abbr: idx for idx, abbr in enumerate(labels)}
Vz_lookup = {idx: name for idx, name in enumerate(labels)}
data['Vz'] = data['VisualBin'].map(Vz_enum).astype('Int64')
data['VzName'] = data['Vz'].map(Vz_lookup)


# Step 2: Group by SizeBin and Type
grouped = data.groupby(['VisualBin', 'Type'], observed=True).size().reset_index(name='Count')

# Step 3: Plot with color by Type
fig = px.bar(grouped, x='VisualBin', y='Count', color='Type',
             title='Object Apparent Mag Distribution by Type',
             category_orders={'VisualBin': labels})  # ensures correct bin order

fig.show()

In [ ]:
bins = [    0,      0.5,      1,      2,      5,       10,       30,        100,      3000]
labels = ['Tiny (<0.5′)', 'Small (0.5–1′)', 'Compact (1–2′)', 'Moderate (2–5′)', 'Prominent (5–10′)', 'Wide (10–30′)', 'Extended (30–100′)', 'Expansive (100′+)']
data['SizeBin'] = pd.cut(data['Size'], bins=bins, labels=labels, include_lowest=True)

Sz_enum = {abbr: idx for idx, abbr in enumerate(labels)}
Sz_lookup = {idx: name for idx, name in enumerate(labels)}
data['Sz'] = data['SizeBin'].map(Sz_enum)
data['SzName'] = data['Sz'].map(Sz_lookup)


# Step 2: Group by SizeBin and Type
grouped = data.groupby(['SizeBin', 'Type'], observed=True).size().reset_index(name='Count')

# Step 3: Plot with color by Type
fig = px.bar(grouped, x='SizeBin', y='Count', color='Type',
             title='Object Size Distribution by Type (Arcmin)',
             category_orders={'SizeBin': labels})  # ensures correct bin order

fig.show()



In [ ]:
Rt_lookup = {
    5: 'Showcase (top 2%)',
    4: 'Excellent (top 10%)',
    3: 'Good (top 25%)',
    2: 'Typical',
    1: 'Challenging',
    0: 'Not recommended'
}
data['Rt'] = data['Rating']
data['RtName'] = data['Rt'].map(Rt_lookup)
grouped = data.groupby(['Rating', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Rating', y='Count', color='Type', title='Object Rating by Type' ) 
fig.show()


In [ ]:
import re
catalogs = [
    ('Messier', 'M'),
    ('Caldwell', 'Caldwell '),
    ('NGC', 'NGC '),
    ('IC', 'IC '),
    ('AbellG', 'Abell '),
    ('AbellN', 'Abell '),
    ('Arp', 'Arp '),
    ('H400', 'H'),
    ('UGC', 'UGC '),
    ('PGC', 'PGC '),
    ('Hickson', 'Hickson '),
    ('Griff.', 'Griffiths '),
    ('Kohou.', 'Kohoutek '),
    ('Mink.', 'Minkowski '),
    ('Barn.', 'Barnard '),
    ('Sh2', 'Sh 2-'),
    ('LBN', 'LBN '),
    ('RCW', 'RCW '),
    ('vdB', 'vdB '),
    ('LDN', 'LDN '),
    ('SNR', 'SNR '),
    ('Gum', 'Gum '),
    ('HT', 'HT '),
    ('SD', 'SD '),
    ('OB', 'OB '),
    ('SP', 'SP '),
    ('FG', 'FG '),
    ('Name', ''),
    ('Object', ''),
]

import re
from collections import OrderedDict

def normalize_catalog_id(text):
    # Remove leading zeros from catalog numbers (e.g. "Abell 01" → "Abell 1")
    text = re.sub(r'(\D+)\s*0+(\d+)', r'\1 \2', str(text))
    # Remove trailing patterns like " (3 of 5)"
    text = re.sub(r'\s*\(\s*\d+\s*of\s*\d+\s*\)', '', text)
    # Rename AbellG to Abell"
    text = re.sub(r'^AbellG\b', 'Abell', text)
    # Remove space in Messier eg M 23 to M23
    text = re.sub(r'^M\s*(\d+)', r'M\1', text)
    # Rename Sh2 to Sh 2
    text = re.sub(r'^Sh2', r'Sh 2', text)
    # Single spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def get_catalog_ids(row):
    primary = None
    nickname = None
    secondary = []
    for col, prefix in catalogs:
        val = row.get(col)
        if pd.notnull(val):
            formatted = f"{prefix}{val}"
            normalized = normalize_catalog_id(formatted)
            if not primary:
                primary = normalized
            else:
                if col=='Name': 
                    nickname = normalized
                elif col!='Object':
                    secondary.append(normalized)
                else:
                    if not (normalized==nickname or normalized==primary):
                        secondary.append(normalized)

    unique_secondary = list(OrderedDict.fromkeys(secondary))
    return pd.Series({ 
        'MainID': primary, 
        'OtherIDs': ', '.join(unique_secondary) if unique_secondary else None,
        'Name': nickname,
    })

data[['MainID', 'OtherIDs', 'Name']] = data.apply(get_catalog_ids, axis=1)

In [ ]:
# Suppliment data
data.loc[data['OtherIDs'] == 'SMC', 'Name'] = 'Small Magellanic Cloud'
data.loc[data['MainID'] == 'LMC', 'Name'] = 'Large Magellanic Cloud'
data.loc[data['MainID'] == 'NGC 2327', 'Name'] = 'Seagull Nebula'
data.loc[data['MainID'] == 'Dark Doodad', 'Name'] = 'Dark Doodad'
data.loc[data['MainID'] == 'Dark Doodad', 'MainID'] = 'NGC 4372'
data.loc[data['MainID'] == 'Caldwell 106', 'Name'] = '47 Tucanae'
data.loc[data['MainID'] == 'vdB 21', 'Name'] = 'Maia Nebula'
data.loc[data['MainID'] == 'vdB 22', 'Name'] = 'Merope Nebula'
data.loc[data['MainID'] == 'vdB 23', 'Name'] = 'Pleiades Nebula'
data.loc[data['MainID'] == 'Caldwell 50', 'Name'] = 'Satellite Cluster'
data.loc[data['MainID'] == 'Caldwell 77', 'Name'] = 'Centaurus A'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'
data.loc[data['MainID'] == 'Chamaeleon I', 'Name'] = 'Chamaeleon dark cloud'
data.loc[data['OtherIDs'] == 'Chamaeleon II', 'Name'] = 'Haast Eagle Nebula, Possum Nebula'
data.loc[data['OtherIDs'] == 'Chamaeleon II', 'MainID'] = 'Chamaeleon II'
data.loc[data['OtherIDs'] == 'Chamaeleon II', 'OtherIDs'] = None
data.loc[data['MainID'] == 'Chamaeleon III', 'Name'] = 'Chamaeleon dark cloud'
data.loc[data['MainID'] == 'LBN 468', 'Name'] = 'Gyulbudaghian\'s Nebula'
data.loc[data['MainID'] == 'LDN 1251', 'Name'] = 'Rotten Fish Nebula'
data.loc[data['MainID'] == 'Abell 262', 'Name'] = 'Galaxy Cluster in Andromeda'
data.loc[data['MainID'] == 'Barnard 7', 'Name'] = 'Taurus Molecular Clouds'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'


In [ ]:
data[['MainID', 'Name', 'VzName', 'C1Name', 'C2Name', 'CnName', 'SzName', 'RtName', 'Notes','OtherIDs']]

In [ ]:
print(C1_lookup)
print(C2_lookup)
print(Cn_lookup)
print(Sz_lookup)
print(Rt_lookup)
print(Vz_lookup)

In [ ]:
import json
selected_columns = ['MainID', 'Name', 'Notes','Class', 'OtherIDs', 'Rt', 'Sz', 'Vz', 'Cn', 'C1', 'C2', 'RA_deg', 'Dec_deg']
filtered_df = data[
    data['Rt'].isin([5, 4, 3]) &
    ~data['Vz'].isin([6, 5]) &
    data['Sz'].isin([7, 6, 5])
][selected_columns]
sorted_df = filtered_df.sort_values(by=['Rt', 'Sz'], ascending=[False, False])
json_str = sorted_df.to_json(orient='records', indent=2)
pathname = '../pilot/public/catalog_a_lg.json'
print(pathname,len(sorted_df))
with open(pathname, 'w') as f:
    f.write(json_str)


filtered_df = data[
    data['Rt'].isin([5, 4, 3]) &
    ~data['Vz'].isin([6, 5]) &
    data['Sz'].isin([4, 3, 2, 1, 0])
][selected_columns]
sorted_df = filtered_df.sort_values(by=['Rt', 'Sz'], ascending=[False, False])
json_str = sorted_df.to_json(orient='records', indent=2)
pathname = '../pilot/public/catalog_a_md.json'
print(pathname,len(sorted_df))
with open(pathname, 'w') as f:
    f.write(json_str)


filtered_df = data[
    data['Rt'].isin([5, 4, 3]) &
    data['Vz'].isin([5])
][selected_columns]
sorted_df = filtered_df.sort_values(by=['Rt', 'Sz'], ascending=[False, False])
json_str = sorted_df.to_json(orient='records', indent=2)
pathname = '../pilot/public/catalog_a_sm.json'
print(pathname,len(sorted_df))
with open(pathname, 'w') as f:
    f.write(json_str)

filtered_df = data[
    data['Rt'].isin([5, 4, 3]) &
    data['Vz'].isin([6])
][selected_columns]
sorted_df = filtered_df.sort_values(by=['Rt', 'Sz'], ascending=[False, False])
json_str = sorted_df.to_json(orient='records', indent=2)
pathname = '../pilot/public/catalog_a_xs.json'
print(pathname,len(sorted_df))
with open(pathname, 'w') as f:
    f.write(json_str)

filtered_df = data[
    data['Rt'].isin([2]) &
    ~data['Vz'].isin([6, 5]) 
][selected_columns]
sorted_df = filtered_df.sort_values(by=['Rt', 'Sz'], ascending=[False, False])
json_str = sorted_df.to_json(orient='records', indent=2)
pathname = '../pilot/public/catalog_b_md.json'
print(pathname,len(sorted_df))
with open(pathname, 'w') as f:
    f.write(json_str)

filtered_df = data[
    data['Rt'].isin([2]) &
    data['Vz'].isin([5])
][selected_columns]
sorted_df = filtered_df.sort_values(by=['Rt', 'Sz'], ascending=[False, False])
json_str = sorted_df.to_json(orient='records', indent=2)
pathname = '../pilot/public/catalog_b_sm.json'
print(pathname,len(sorted_df))
with open(pathname, 'w') as f:
    f.write(json_str)


filtered_df = data[
    data['Rt'].isin([2]) &
    data['Vz'].isin([6])
][selected_columns]
sorted_df = filtered_df.sort_values(by=['Rt', 'Sz'], ascending=[False, False])
json_str = sorted_df.to_json(orient='records', indent=2)
pathname = '../pilot/public/catalog_b_xs.json'
print(pathname,len(sorted_df))
with open(pathname, 'w') as f:
    f.write(json_str)



In [ ]:
import numpy as np
from datetime import datetime, timezone

a='░▁▂▃▄▅▆▇█▔░▁_▂▃▄▅▆▇▔'

def categorize_alt(alt):
    if alt < 0:
        return 'Below Horizon'  # Not visible
    elif alt < 12:
        return 'Near Horizon'   # Rising or setting, poor visibility
    elif alt < 30:
        return 'Low Altitude'   # Often affected by atmospheric distortion
    elif alt < 60:
        return 'Mid Altitude'   # Good visibility, moderate elevation
    elif alt < 82:
        return 'High Altitude'  # Excellent visibility, optimal for imaging
    else:
        return 'Near Zenith'    # Peak elevation, polaris cannot reach


def approx_altaz(ra_deg, dec_deg, observer_lat, observer_lon, time_utc):
    # Convert to radians
    ra = np.radians(ra_deg)
    dec = np.radians(dec_deg)
    lat = np.radians(observer_lat)

    # Julian Date
    jd = (time_utc - datetime(2000, 1, 1, tzinfo=timezone.utc)).total_seconds() / 86400.0 + 2451545.0

    # Local Sidereal Time (LST) in degrees
    lst_deg = (100.46 + 0.985647 * (jd - 2451545.0) + observer_lon + np.degrees(ra)) % 360
    ha_rad = np.radians(lst_deg - ra_deg)  # Hour angle in radians

    # Altitude
    alt_rad = np.arcsin(np.sin(lat) * np.sin(dec) + np.cos(lat) * np.cos(dec) * np.cos(ha_rad))
    alt_deg = np.degrees(alt_rad)

    # Azimuth
    cz = (np.sin(dec) - np.sin(lat) * np.sin(alt_rad)) / (np.cos(lat) * np.cos(alt_rad))
    cz = np.clip(cz, -1, 1)  # Avoid domain errors
    az_rad = np.arccos(cz)
    az_deg = np.degrees(az_rad)
    az_deg = np.where(np.sin(ha_rad) < 0, az_deg, 360 - az_deg)

    return az_deg, alt_deg
